_한국어로 기계번역됨_


# 그래프에서 LLM 토큰 스트리밍하는 방법

!!! 정보 "전제 조건"

이 가이드는 다음 내용을 잘 알고 있다고 가정합니다.

- [스트리밍](../../concepts/streaming/)
- [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)

LangGraph로 LLM 애플리케이션을 구축할 때, LangGraph 노드 내에서 LLM 호출로부터 개별 LLM 토큰을 스트리밍하고 싶을 수 있습니다. 이를 `graph.stream(..., stream_mode="messages")`를 통해 수행할 수 있습니다:

```python
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
def call_model(state: State):
    model.invoke(...)
    ...

graph = (
    StateGraph(State)
    .add_node(call_model)
    ...
    .compile()
    
for msg, metadata in graph.stream(inputs, stream_mode="messages"):
    print(msg)
```

스트리밍된 출력은 `(메시지 청크, 메타데이터)` 형식의 튜플입니다:

* 메시지 청크는 LLM이 스트리밍한 토큰입니다.
* 메타데이터는 LLM이 호출된 그래프 노드에 대한 정보와 LLM 호출 메타데이터가 포함된 딕셔너리입니다.

!!! 노트 "LangChain 없이 사용하기"

LLM 토큰을 **LangChain 없이 스트리밍해야 하는 경우**, [`stream_mode="custom"`](../streaming/#custom)를 사용하여 LLM 공급자 클라이언트로부터 직접 출력을 스트리밍할 수 있습니다. 더 많은 정보를 학습하려면 [아래 예시](#example-without-langchain)를 확인하십시오.

!!! 경고 "Python < 3.11의 비동기 처리"

Python < 3.11에서 비동기 코드를 사용할 때는, 호출할 때 `RunnableConfig`를 채팅 모델에 수동으로 전달해야 합니다: `model.ainvoke(..., config)`.
스트림 메서드는 콜백으로 전달된 스트리밍 트레이서를 사용하여 중첩된 코드에서 모든 이벤트를 수집합니다. 3.11 이상에서는 [contextvars](https://docs.python.org/3/library/contextvars.html)를 통해 자동으로 처리되지만, 3.11 이전에는 [asyncio의 작업](https://docs.python.org/3/library/asyncio-task.html#asyncio.create_task)이 적절한 `contextvar` 지원이 부족하여 콜백이 수동으로 구성(config)을 전달해야만 전파될 수 있습니다. 이는 아래의 `call_model` 함수에서 수행됩니다.


## 설정

먼저 필요한 패키지를 설치해야 합니다.


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai


다음으로, OpenAI(우리가 사용할 LLM)에 대한 API 키를 설정해야 합니다.


In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축된 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 읽어보세요.
    </p>
</div>


!!! 주의 사항 수동 콜백 전파

    아래의 `call_model(state: State, config: RunnableConfig):`에서, a) 노드 함수에서 [`RunnableConfig`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.config.RunnableConfig.html#langchain_core.runnables.config.RunnableConfig)를 받고, b) `model.ainvoke(..., config)`의 두 번째 인자로 전달합니다. 이는 파이썬 3.11 이상에서는 선택 사항입니다.


## 예제


아래에서는 단일 노드에서 두 개의 LLM 호출을 포함한 예제를 보여드립니다.


In [3]:
from typing import TypedDict
from langgraph.graph import START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI


# Note: we're adding the tags here to be able to filter the model outputs down the line
joke_model = ChatOpenAI(model="gpt-4o-mini", tags=["joke"])
poem_model = ChatOpenAI(model="gpt-4o-mini", tags=["poem"])


class State(TypedDict):
    topic: str
    joke: str
    poem: str


# highlight-next-line
async def call_model(state, config):
    topic = state["topic"]
    print("Writing joke...")
    # Note: Passing the config through explicitly is required for python < 3.11
    # Since context var support wasn't added before then: https://docs.python.org/3/library/asyncio-task.html#creating-tasks
    joke_response = await joke_model.ainvoke(
        [{"role": "user", "content": f"Write a joke about {topic}"}],
        # highlight-next-line
        config,
    )
    print("\n\nWriting poem...")
    poem_response = await poem_model.ainvoke(
        [{"role": "user", "content": f"Write a short poem about {topic}"}],
        # highlight-next-line
        config,
    )
    return {"joke": joke_response.content, "poem": poem_response.content}


graph = StateGraph(State).add_node(call_model).add_edge(START, "call_model").compile()


In [4]:
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    # highlight-next-line
    stream_mode="messages",
):
    if msg.content:
        print(msg.content, end="|", flush=True)


Writing joke...
Why| was| the| cat| sitting| on| the| computer|?

|Because| it| wanted| to| keep| an| eye| on| the| mouse|!|

Writing poem...
In| sun|lit| patches|,| sleek| and| sly|,|  
|Wh|isk|ers| twitch| as| shadows| fly|.|  
|With| velvet| paws| and| eyes| so| bright|,|  
|They| dance| through| dreams|,| both| day| and| night|.|  

|A| playful| p|ounce|,| a| gentle| p|urr|,|  
|In| every| leap|,| a| soft| allure|.|  
|Cur|led| in| warmth|,| a| silent| grace|,|  
|Each| furry| friend|,| a| warm| embrace|.|  

|Myst|ery| wrapped| in| fur| and| charm|,|  
|A| soothing| presence|,| a| gentle| balm|.|  
|In| their| gaze|,| the| world| slows| down|,|  
|For| in| their| realm|,| we're| all| ren|own|.|

In [5]:
metadata


{'langgraph_step': 1,
 'langgraph_node': 'call_model',
 'langgraph_triggers': ['start:call_model'],
 'langgraph_path': ('__pregel_pull', 'call_model'),
 'langgraph_checkpoint_ns': 'call_model:6ddc5f0f-1dd0-325d-3014-f949286ce595',
 'checkpoint_ns': 'call_model:6ddc5f0f-1dd0-325d-3014-f949286ce595',
 'ls_provider': 'openai',
 'ls_model_name': 'gpt-4o-mini',
 'ls_model_type': 'chat',
 'ls_temperature': 0.7,
 'tags': ['poem']}

### 특정 LLM 호출로 필터링


우리는 모든 LLM 호출에서 토큰을 스트리밍하고 있음을 알 수 있습니다. 이제 스트리밍된 토큰을 특정 LLM 호출만 포함하도록 필터링해 보겠습니다. 우리는 스트리밍된 메타데이터를 사용하고, 이전에 LLM에 추가한 태그를 사용하여 이벤트를 필터링할 수 있습니다:


In [6]:
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    stream_mode="messages",
):
    # highlight-next-line
    if msg.content and "joke" in metadata.get("tags", []):
        print(msg.content, end="|", flush=True)


Writing joke...
Why| was| the| cat| sitting| on| the| computer|?

|Because| it| wanted| to| keep| an| eye| on| the| mouse|!|

Writing poem...


## LangChain 없이 예시


In [7]:
from openai import AsyncOpenAI

openai_client = AsyncOpenAI()
model_name = "gpt-4o-mini"


async def stream_tokens(model_name: str, messages: list[dict]):
    response = await openai_client.chat.completions.create(
        messages=messages, model=model_name, stream=True
    )

    role = None
    async for chunk in response:
        delta = chunk.choices[0].delta

        if delta.role is not None:
            role = delta.role

        if delta.content:
            yield {"role": role, "content": delta.content}


# highlight-next-line
async def call_model(state, config, writer):
    topic = state["topic"]
    joke = ""
    poem = ""

    print("Writing joke...")
    async for msg_chunk in stream_tokens(
        model_name, [{"role": "user", "content": f"Write a joke about {topic}"}]
    ):
        joke += msg_chunk["content"]
        metadata = {**config["metadata"], "tags": ["joke"]}
        chunk_to_stream = (msg_chunk, metadata)
        # highlight-next-line
        writer(chunk_to_stream)

    print("\n\nWriting poem...")
    async for msg_chunk in stream_tokens(
        model_name, [{"role": "user", "content": f"Write a short poem about {topic}"}]
    ):
        poem += msg_chunk["content"]
        metadata = {**config["metadata"], "tags": ["poem"]}
        chunk_to_stream = (msg_chunk, metadata)
        # highlight-next-line
        writer(chunk_to_stream)

    return {"joke": joke, "poem": poem}


graph = StateGraph(State).add_node(call_model).add_edge(START, "call_model").compile()


!!! 노트 "stream_mode="custom""

    LangChain 없이 LLM 토큰을 스트리밍할 때, [`stream_mode="custom"`](../streaming/#stream-modecustom)를 사용하는 것을 권장합니다. 이를 통해 LangGraph 스트리밍 출력에 포함할 LLM 공급자 API의 데이터를 명시적으로 제어할 수 있으며, 추가 메타데이터를 포함시킬 수 있습니다.


In [8]:
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    # highlight-next-line
    stream_mode="custom",
):
    print(msg["content"], end="|", flush=True)


Writing joke...
Why| was| the| cat| sitting| on| the| computer|?

|Because| it| wanted| to| keep| an| eye| on| the|

Writing poem...
 mouse|!|In| sun|lit| patches|,| they| stretch| and| y|awn|,|  
|With| whispered| paws| at| the| break| of| dawn|.|  
|Wh|isk|ers| twitch| in| the| morning| light|,|  
|Sil|ken| shadows|,| a| graceful| sight|.|  

|The| gentle| p|urr|s|,| a| soothing| song|,|  
|In| a| world| of| comfort|,| where| they| belong|.|  
|M|yster|ious| hearts| wrapped| in| soft|est| fur|,|  
|F|eline| whispers| in| every| p|urr|.|  

|Ch|asing| dreams| on| a| moon|lit| chase|,|  
|With| a| flick| of| a| tail|,| they| glide| with| grace|.|  
|Oh|,| playful| spirits| of| whisk|ered| cheer|,|  
|In| your| quiet| company|,| the| world| feels| near|.|  |

In [9]:
metadata


{'langgraph_step': 1,
 'langgraph_node': 'call_model',
 'langgraph_triggers': ['start:call_model'],
 'langgraph_path': ('__pregel_pull', 'call_model'),
 'langgraph_checkpoint_ns': 'call_model:3fa3fbe1-39d8-5209-dd77-0da38d4cc1c9',
 'tags': ['poem']}

특정 LLM 호출을 필터링하려면 스트리밍 메타데이터를 사용할 수 있습니다:


In [10]:
async for msg, metadata in graph.astream(
    {"topic": "cats"},
    stream_mode="custom",
):
    # highlight-next-line
    if "poem" in metadata.get("tags", []):
        print(msg["content"], end="|", flush=True)


Writing joke...


Writing poem...
In| shadows| soft|,| they| weave| and| play|,|  
|With| whispered| paws|,| they| greet| the| day|.|  
|Eyes| like| lantern|s|,| bright| and| keen|,|  
|Guard|ians| of| secrets|,| unseen|,| serene|.|  

|They| twist| and| stretch| in| sun|lit| beams|,|  
|Ch|asing| the| echoes| of| half|-|formed| dreams|.|  
|With| p|urring| songs| that| soothe| the| night|,|  
|F|eline| spirits|,| pure| delight|.|  

|On| windows|ills|,| they| perch| and| stare|,|  
|Ad|vent|urers| bold| with| a| graceful| flair|.|  
|In| every| leap| and| playful| bound|,|  
|The| magic| of| cats|—|where| love| is| found|.|